# Step 2: ANALYZE

In [1]:
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import re
import pandas as pd
import numpy as np
import platform

In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as SW_en
from spacy.lang.es.stop_words import STOP_WORDS as SW_es

In [3]:
from collections import Counter

In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [5]:
#import nltk 
#from nltk.corpus import stopwords 
#set(stopwords.words('english'))


In [6]:
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_sm
#!python -m spacy download es_core_news_md

In [8]:
dic_key_words = \
{   'data analyst'  : ['data analyst','analisis de datos', 
                       'data analytics', 'analitica de datos',
                       'business analyst', 'analisis de negocios',
                       'sql',
                       'python',
                       'business intelligence', 'inteligencia de negocios' , ' bi ' , 
                       'excel', 'microsoft office', 'ms office' ,
                       'dashboard','kpi', 
                       'reporting', 'informes',
                       'tableau', 'powerbi', 'power bi', 'microsoft bi', 'qlik', 'spotfire',
                      ],
 
    'data scientist': ['data scientist', 'cientifico de datos',
                       'data science', 'ciencia de datos',
                       'sql', 
                       'python', ' r ', 
                       ' ml ', 'machine learning','deep learning','neuronal networks', 
                       'pandas', 'numpy', 'scipy', 'web scrapping' , 
                       'data visualization', 'visualizacion de datos','matplotlib', 'seaborn', 
                       'spss',  'statistics',
                       'sklearn','sk learn', 'sk-learn','scikit learn', 'scikit-learn' ,'tensorflow', 'keras',
                       'data wrangling', 'limpieza de datos',
                       ' nlp ', 'a/b testing','a b testing'
                      ],
 
    'data engineer':  ['data engineer', 'ingeniero de datos',
                       'sql', 'nosql', 'no sql', 'bases de datos no relacionales',
                       'python', ' r ', 'scala', 'java ', 'spark', 'hadoop', 'haddop' 'hive', 'impala', 'kafka', 
                       'data wrangling', 'limpieza de datos',
                       'etl','extract, transform and load','map reduce','mapreduce', 
                       'data acquisition', 'adquisicion de datos',
                       'big data', 'bigdata', 'bigquery',
                       'data warehouse', 
                       ' aws ', 'azure',
                       'software engineering'
                      ]
}

## Getting csv

#### > PROFILE:

In [9]:
platform.system()

'Linux'

In [10]:
if platform.system() == 'Linux':
    df_profile = pd.read_csv(f'../data/raw/df_profile_cristopherrojaslepe.csv',
                             sep = ';',
                             encoding= 'utf8'
                              )
    
elif platform.system() == 'Windows':
    df_profile = pd.read_csv(f'..\\data\\raw\\df_profile.csv',
                             sep = ';',
                             encoding= 'utf8'
                            )
    
df_profile

,Unnamed: 0,info
0,profile,cristopherrojaslepe
1,headline,data scientist & analyst | python | pandas | n...
2,open new jobs,True
3,about,data scientist & analyst and industrial engine...
4,experiences,"['collection coordination analyst', 'real esta..."
5,years of experiences,"['1 yr 2 mos', '1 yr', '2 yrs 1 mo', '8 mos', ..."
6,total years,7
7,degrees,"['master', 'engineering - be']"
8,fields,"['data analytics', 'industrial engineering']"
9,skills,"['microsoft project', 'español', 'microsoft of..."


In [11]:
## ONLY FOR JUPYTER !!!!!!!!!!
df_profile.rename(columns={'Unnamed: 0':'type'},inplace=True)
df_profile.set_index('type', inplace=True)

In [12]:
df_profile

,info
type,
profile,cristopherrojaslepe
headline,data scientist & analyst | python | pandas | n...
open new jobs,True
about,data scientist & analyst and industrial engine...
experiences,"['collection coordination analyst', 'real esta..."
years of experiences,"['1 yr 2 mos', '1 yr', '2 yrs 1 mo', '8 mos', ..."
total years,7
degrees,"['master', 'engineering - be']"
fields,"['data analytics', 'industrial engineering']"


#### > JOBS:

In [47]:

#f'../data/raw/df_jobs_Madrid_2020-04-18 23.50.csv',
#f'..\\data\\raw\\df_jobs_Madrid_2020-04-18 23.50.csv'
if platform.system() == 'Linux':
    df_jobs_raw = pd.read_csv(f'../data/raw/df_jobs_Barcelona_2020-04-21 00.58.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )
elif platform.system() == 'Windows':
    df_jobs_raw = pd.read_csv(f'..\\data\\raw\\df_jobs_Barcelona_2020-04-21 00.58.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )

In [48]:
df_jobs = df_jobs_raw

In [49]:
df_jobs.shape

(180, 18)

In [50]:
df_jobs.head(3)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Job Description,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info
0,data analyst,Barcelona,2020-04-21 00:58,1753506830,https://www.linkedin.com/jobs/view/1753506830/,Solution Architect-Applied Intelligence Barcelona,Accenture España,"Barcelona, ES",3 weeks,2020-03-31 00:58,False,Accenture Digital offers a comprehensive portf...,"{'yes': [], 'no': ['Serverless Computing', 'Di...",Associate,Information Technology & Services Computer Sof...,Full-time,Engineering Information Technology,Posted Date\nPosted 3 weeks ago\nNumber of vie...
1,data analyst,Barcelona,2020-04-21 00:58,1816757840,https://www.linkedin.com/jobs/view/1816757840/,Senior Investment Accounting Analyst.,Zurich Insurance Company Ltd,"Barcelona, ES",1 week,2020-04-14 00:58,False,Our opportunity\n\nInvestment Administration &...,"{'yes': [], 'no': ['Serverless Computing', 'Di...",NaN,Financial Services Insurance,Full-time,Accounting/Auditing Finance,Posted Date\nPosted 1 week ago\nNumber of view...
2,data analyst,Barcelona,2020-04-21 00:58,1769986744,https://www.linkedin.com/jobs/view/1769986744/,Accounts Payable Analyst - Dutch Speaker,Tech Data,"Barcelona, ES",2 weeks,2020-04-07 00:58,False,It's rewarding to work at a company where YOU ...,"{'yes': [], 'no': ['Serverless Computing', 'Di...",NaN,Computer Hardware Computer Software Informatio...,Full-time,Accounting/Auditing Finance,Posted Date\nPosted 2 weeks ago\nNumber of vie...


In [51]:
# ONLY IN CASE TO CORRECT WRONG DF
#df_jobs.drop(columns='Job htmlJob name', inplace=True)

#df_jobs = df_jobs[['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
#       'Job html', 'Job name', 'Company name', 'Company location',
#       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
#       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
#       'Job info']]

In [52]:
df_jobs[df_jobs['JOB TITLE']=='data analyst'].shape

(62, 18)

In [53]:
df_jobs[df_jobs['JOB TITLE']=='data scientist'].shape

(90, 18)

In [54]:
df_jobs[df_jobs['JOB TITLE']=='data engineer'].shape

(28, 18)

## Analizyng info

### > Cleaning jobs:

In [55]:
df_jobs.loc[0,'JOB TITLE':'Current Job Id']

JOB TITLE              data analyst
LOCATION                  Barcelona
SEARCH DATETIME    2020-04-21 00:58
Current Job Id           1753506830
Name: 0, dtype: object

##### * JOB TITLE_ID

In [56]:
df_jobs['JOB TITLE_ID'] = df_jobs.apply(lambda x: x['JOB TITLE'] +"_"+ str(x['Current Job Id']), axis = 1 )

In [57]:
df_jobs.shape[0] 

180

In [58]:
len(df_jobs['JOB TITLE_ID'].unique())

180

In [59]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['JOB TITLE_ID'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
JOB TITLE_ID,
data analyst_1422096341,1
data scientist_1758546757,1
data scientist_1768580522,1
data scientist_1772485017,1
data scientist_1776631970,1
...,...
data analyst_1830200694,1
data engineer_1117356782,1
data engineer_1401777412,1


In [60]:
df_jobs[df_jobs['Current Job Id']==1774326710]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Job Description,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID


In [63]:
df_jobs.shape[0] - len(df_jobs['JOB TITLE_ID'].unique())

0

In [64]:
df_jobs.drop_duplicates(subset='JOB TITLE_ID', keep='first', inplace = True)

In [65]:
df_jobs.shape

(180, 19)

##### * Current Job Id

In [30]:
len(df_jobs['Current Job Id'].unique())

638

In [31]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['Current Job Id'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
Current Job Id,
1794127974,3
1828401888,3
1764841143,3
1768494778,3
1757263851,3
...,...
1794130353,1
1793255231,1
1793229309,1


In [66]:
df_jobs[df_jobs['Current Job Id']==1645918674]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Job Description,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID


### > Language of each job description:

In [67]:
df_jobs['Job name'].head(10)

0    Solution Architect-Applied Intelligence Barcelona
1                Senior Investment Accounting Analyst.
2             Accounts Payable Analyst - Dutch Speaker
3                          Data Analyst (sector Farma)
4                                  IT Business Analyst
5    Computational Linguist with Spanish - Mexican ...
6                                  Master Data Analyst
7                       Consultor Data Analytics (f/m)
8                            Data Analytics Consultant
9                                         Data Analyst
Name: Job name, dtype: object

In [69]:
s = df_jobs.loc[6,'Job Description']
s

"It's rewarding to work at a company where YOU make a difference! If you're passionate about the ever-changing IT industry and enjoy working in a fast-paced, collaborative progressive environment, you'll thrive at Tech Data! Join our team to connect the world with the power of technology!\nTech Data es uno de los mayoristas más importantes del mundo en productos, servicios y soluciones de tecnología.\nSus recursos avanzados de logística y sus servicios de valor añadido ayudan a más de 115.000 proveedores de IT a dar soporte a las necesidades tecnológicas de los usuarios de más de 100 países.\nTech Data es un distribuidor de productos multitecnología para diferentes sectores del mundo empresarial: desde Enterprise (Azlan) a movilidad (Tech Data Mobile), desde Audio Video Profesional (Maverick) hasta Software de diseño (Datech), desde la tradicional IT (Tech Data) hasta consumibles electrónicos.\n\nTech Data está creciendo cada día más y buscamos incorporar un/a Master Data Analyst o gra

In [70]:
detect(s)

'es'

In [71]:
df_jobs['Info Language'] = df_jobs['Job info'].apply(lambda x: detect(x))

In [72]:
df_jobs['Info Language'].unique()

array(['en', 'es'], dtype=object)

In [73]:
df_jobs['Seniority Level'].unique()

array(['Associate', nan, 'Entry level', 'Mid-Senior level', 'Executive',
       'Internship', 'Director'], dtype=object)

### > Coincidences with definitions:

In [74]:
def coincidencias(job_skills, post_skills):
    c = 0
    for i in job_skills: 
        if re.search(i,post_skills): 
            c+=1
    return c
        

In [75]:
def list_coincidencias(job_skills, post_skills):
    l=[]
    for i in job_skills:
        if re.search(i,post_skills):
            l.append(i)
    return l
        

In [76]:
def clean_text(text):
    text = text.replace('\n',' ')
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.replace('.',' ')
    text = text.lower()
    
    return text
    

#### jobs >

In [79]:
jobs_list = list(df_jobs['JOB TITLE'].unique()) #only jobs that searcht has been taken

In [81]:

for j in jobs_list:
    skills_list = dic_key_words.get(j) #dictionary

    df_jobs[f'{j[0:2]} coincidences'] = \
    df_jobs['Job info'].apply(lambda x: coincidencias(skills_list,clean_text(x)))
    df_jobs[f'{j[0:2]} coincidences_list'] = \
    df_jobs['Job info'].apply(lambda x: list_coincidencias(skills_list,clean_text(x)))

#     df_jobs['DS coincidences'] = \
#     df_jobs['Job info'].apply(lambda x: coincidencias(DS_list,clean_text(x)))
#     df_jobs['DS coincidences_list'] = \
#     df_jobs['Job info'].apply(lambda x: list_coincidencias(DS_list,clean_text(x)))

#     df_jobs['DE coincidences'] = \
#     df_jobs['Job info'].apply(lambda x: coincidencias(DE_list,clean_text(x)))
#     df_jobs['DE coincidences_list'] = \
#     df_jobs['Job info'].apply(lambda x: list_coincidencias(DE_list,clean_text(x)))

#### profile >

In [82]:
df_profile.loc['total skills', 'info']

"['analytical skills', 'bash', 'business analysis', 'business intelligence', 'business process improvement', 'continuous improvement', 'data analysis', 'data analytics', 'data modeling', 'data pipelines', 'data science', 'data visualization', 'engineering', 'engineering - be', 'english', 'español', 'financial analysis', 'git', 'industrial engineering', 'industry knowledge', 'jupyter', 'leadership', 'lean manufacturing', 'machine learning', 'master', 'microsoft excel', 'microsoft office', 'microsoft power bi', 'microsoft power query', 'microsoft project', 'mysql', 'negotiation', 'numpy', 'pandas', 'power pivot', 'process engineering', 'process optimization', 'programming', 'project management', 'python', 'r', 'sap', 'scrum', 'seaborn', 'six sigma', 'spanish', 'sql', 'statistics', 'tableau', 'team management', 'teamwork', 'telco', 'vba', 'visual basic for applications', 'web scraping']"

In [83]:
#### THIS IS ONLY FOR JUPYTER !!!!!!!!!!!!!!!!!!!!
profile_skills = clean_text(df_profile.loc['skills', 'info']).replace('[','').replace(']','').replace("'","").split(', ')
sorted(profile_skills)

['analytical skills',
 'bash',
 'business analysis',
 'business intelligence',
 'business process improvement',
 'continuous improvement',
 'data analysis',
 'data analytics',
 'data modeling',
 'data pipelines',
 'data science',
 'data visualization',
 'engineering',
 'english',
 'español',
 'financial analysis',
 'git',
 'industry knowledge',
 'jupyter',
 'leadership',
 'lean manufacturing',
 'machine learning',
 'microsoft excel',
 'microsoft office',
 'microsoft power bi',
 'microsoft power query',
 'microsoft project',
 'mysql',
 'negotiation',
 'numpy',
 'pandas',
 'power pivot',
 'process engineering',
 'process optimization',
 'programming',
 'project management',
 'python',
 'r',
 'sap',
 'scrum',
 'seaborn',
 'six sigma',
 'sql',
 'statistics',
 'tableau',
 'team management',
 'teamwork',
 'telco',
 'vba',
 'visual basic for applications',
 'web scraping']

In [84]:
df_jobs['PROFILE coincidences'] = \
df_jobs['Job info'].apply(lambda x:      coincidencias(profile_skills,clean_text(x)))
df_jobs['PROFILE coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(profile_skills,clean_text(x)))

In [88]:
df_jobs.head(2)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID,Info Language,da coincidences,da coincidences_list,PROFILE coincidences,PROFILE coincidences_list
0,data analyst,Barcelona,2020-04-21 00:58,1753506830,https://www.linkedin.com/jobs/view/1753506830/,Solution Architect-Applied Intelligence Barcelona,Accenture España,"Barcelona, ES",3 weeks,2020-03-31 00:58,...,Information Technology & Services Computer Sof...,Full-time,Engineering Information Technology,Posted Date\nPosted 3 weeks ago\nNumber of vie...,data analyst_1753506830,en,6,"[sql, nosql, python, java , big data, azure]",9,"[english, git, sql, programming, python, r, en..."
1,data analyst,Barcelona,2020-04-21 00:58,1816757840,https://www.linkedin.com/jobs/view/1816757840/,Senior Investment Accounting Analyst.,Zurich Insurance Company Ltd,"Barcelona, ES",1 week,2020-04-14 00:58,...,Financial Services Insurance,Full-time,Accounting/Auditing Finance,Posted Date\nPosted 1 week ago\nNumber of view...,data analyst_1816757840,en,0,[],3,"[sap, english, r]"


#### checking results >

In [85]:
check = 10

In [86]:
df_jobs.loc[check ,'Job name']

'Data Analyst Fotocasa'

In [89]:
dict(df_jobs.loc[check ,'da coincidences':'PROFILE coincidences_list'])

{'da coincidences': 2,
 'da coincidences_list': ['sql', ' r '],
 'PROFILE coincidences': 4,
 'PROFILE coincidences_list': ['git', 'sql', 'r', 'tableau']}

In [90]:
clean_text(df_jobs.loc[check ,'Job Description'])

'descripcion del empleo  estamos buscando un/a excepcional data/insight analyst para formar parte del equipo de data & insights de fotocasa  trabajaras como un miembro mas del equipo de data & insights, colaborando mano a mano con diferentes departamentos como producto, marketing o business development  tu mision principal es la de definir y realizar analisis de negocio y muy especialmente de producto y marketing, ayudando a entender el comportamiento de nuestros usuarios y clientes y asi mejorar el servicio que les ofrecemos   estas seran tus funciones identificar y comprender las necesidades de analisis de producto, marketing y business development para definir una solucion con los resuelva  analizar e interpretar kpis de nuestros sites entendiendo el comportamiento de usuarios, identificando tendencias, riesgos y oportunidades para dar insights que ayuden a realizar recomendaciones para optimizar y mejorar el servicio  a/b testing: participaras en la definicion de test, desde la cre

### TOP JOB POST >

In [91]:
top = 5

In [92]:
df_jobs.sort_values(by=['PROFILE coincidences'], ascending=False).head(top)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID,Info Language,da coincidences,da coincidences_list,PROFILE coincidences,PROFILE coincidences_list
152,data engineer,Barcelona,2020-04-21 00:58,1682047755,https://www.linkedin.com/jobs/view/1682047755/,Lead Data Scientist,TeamCMP,"Barcelona, ES",6 days,2020-04-15 00:58,...,Internet,Full-time,Information Technology Analyst,Posted Date\nPosted 6 days ago\nNumber of view...,data engineer_1682047755,en,6,"[data engineer, sql, python, r , spark, aws ]",15,"[data analysis, english, git, sql, data scienc..."
62,data scientist,Barcelona,2020-04-21 00:58,1682047755,https://www.linkedin.com/jobs/view/1682047755/,Lead Data Scientist,TeamCMP,"Barcelona, ES",6 days,2020-04-15 00:58,...,Internet,Full-time,Information Technology Analyst,Posted Date\nPosted 6 days ago\nNumber of view...,data scientist_1682047755,en,6,"[data engineer, sql, python, r , spark, aws ]",15,"[data analysis, english, git, sql, data scienc..."
143,data scientist,Barcelona,2020-04-21 00:58,1816580513,https://www.linkedin.com/jobs/view/1816580513/,Data Scientist Internship,Entravision,"Barcelona, ES",3 weeks,2020-03-31 00:58,...,Marketing & Advertising Online Media Internet,Internship,Engineering Information Technology,Posted Date\nPosted 3 weeks ago\nNumber of vie...,data scientist_1816580513,en,5,"[sql, python, scala, spark, aws ]",14,"[english, git, sql, data visualization, data s..."
123,data scientist,Barcelona,2020-04-21 00:58,1780265203,https://www.linkedin.com/jobs/view/1780265203/,Sr Data Scientist,Entravision,"Barcelona, ES",1 month,2020-03-24 00:58,...,Marketing & Advertising Online Media Internet,Full-time,Other,Posted Date\nPosted 1 month ago\nNumber of vie...,data scientist_1780265203,en,6,"[sql, python, scala, java , spark, aws ]",14,"[english, git, sql, data visualization, data s..."
135,data scientist,Barcelona,2020-04-21 00:58,1756897291,https://www.linkedin.com/jobs/view/1756897291/,Sr Data Scientist,Entravision Communications,"Barcelona, ES",2 months,2020-02-25 00:58,...,Marketing & Advertising Online Media Internet,Full-time,Other,Posted Date\nPosted 2 months ago\nNumber of vi...,data scientist_1756897291,en,6,"[sql, python, scala, java , spark, aws ]",14,"[english, git, sql, data visualization, data s..."


In [94]:
df_jobs[df_jobs['Current Job Id']==1682047755]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID,Info Language,da coincidences,da coincidences_list,PROFILE coincidences,PROFILE coincidences_list
62,data scientist,Barcelona,2020-04-21 00:58,1682047755,https://www.linkedin.com/jobs/view/1682047755/,Lead Data Scientist,TeamCMP,"Barcelona, ES",6 days,2020-04-15 00:58,...,Internet,Full-time,Information Technology Analyst,Posted Date\nPosted 6 days ago\nNumber of view...,data scientist_1682047755,en,6,"[data engineer, sql, python, r , spark, aws ]",15,"[data analysis, english, git, sql, data scienc..."
152,data engineer,Barcelona,2020-04-21 00:58,1682047755,https://www.linkedin.com/jobs/view/1682047755/,Lead Data Scientist,TeamCMP,"Barcelona, ES",6 days,2020-04-15 00:58,...,Internet,Full-time,Information Technology Analyst,Posted Date\nPosted 6 days ago\nNumber of view...,data engineer_1682047755,en,6,"[data engineer, sql, python, r , spark, aws ]",15,"[data analysis, english, git, sql, data scienc..."


## Using NLP

In [95]:
nlp_en = spacy.load('en_core_web_md')
nlp_es = spacy.load('es_core_news_md')

In [96]:
df_jobs.columns

Index(['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
       'Job html', 'Job name', 'Company name', 'Company location',
       'Posted date', 'Estimated post date', 'Easy apply', 'Job Description',
       'Skills match', 'Seniority Level', 'Industry', 'Employment Type',
       'Job Functions', 'Job info', 'JOB TITLE_ID', 'Info Language',
       'da coincidences', 'da coincidences_list', 'PROFILE coincidences',
       'PROFILE coincidences_list'],
      dtype='object')

In [97]:
unique_jobs = list(df_jobs['JOB TITLE'].unique())
unique_jobs

['data analyst', 'data scientist', 'data engineer']

In [98]:
unique_jobs[0].split(" ")[0][0].upper() + unique_jobs[0].split(" ")[1][0].upper()

'DA'

In [99]:
unique_languages = list(df_jobs['Info Language'].unique())
unique_languages

['en', 'es']

In [161]:
location = df_jobs['LOCATION'].unique()[0].lower()
location

'barcelona'

In [163]:
dict_corpus = {}
dict_filtered_tokens = {}

In [164]:
for job in unique_jobs:
    job_key = job.split(" ")[0][0] + job.split(" ")[1][0]
    job_key = job_key.upper()
    
    for lang in unique_languages:
        
        #CORPUS
        filter_job  = df_jobs['JOB TITLE'] == job
        filter_lang = df_jobs['Info Language'] == lang
        corpus = df_jobs['Job Description'][filter_job & filter_lang].str.cat(sep=" || ")
        
        dict_corpus.update( {f'{job_key}_{lang}' : corpus} )
        
        
        #TOKENS
        if   lang == 'en':
            tokens = nlp_en(corpus)
            STOP_WORDS = SW_en
        elif lang == 'es':
            tokens = nlp_es(corpus)
            STOP_WORDS = SW_es

        filtered_tokens = []

        for word in tokens:
            lemma = word.lemma_.lower().strip()

            if lemma not in STOP_WORDS and re.search('^[a-z]+$', lemma) and lemma not in black_list and len(lemma)>1:
                    filtered_tokens.append(lemma)
        
        dict_filtered_tokens.update( {f'{job_key}_{lang}': filtered_tokens} )

In [165]:
dict_corpus.keys()

dict_keys(['DA_en', 'DA_es', 'DS_en', 'DS_es', 'DE_en', 'DE_es'])

In [166]:
dict_corpus.get('DA_en')[400:600]

' Digital, help our clients grow their business in entirely new ways. Artificial Intelligence and Analytics enables our clients to achieve high performance through insights from data - insights that in'

In [167]:
dict_filtered_tokens.keys()

dict_keys(['DA_en', 'DA_es', 'DS_en', 'DS_es', 'DE_en', 'DE_es'])

In [168]:
#dict_filtered_tokens.get('DA_en')[600:650]

### Counting frequent words

In [171]:
word_freq = Counter(dict_filtered_tokens.get('DA_en'))

In [172]:
black_list = \
[location,
 'empresa','work','skill',
'parir','comer','timar',
 'job','industry', 'type','employment', 'full','functions','seniority','level',
]

In [173]:
common_words = word_freq.most_common(30)
common_words

[('work', 144),
 ('datum', 123),
 ('team', 123),
 ('job', 114),
 ('business', 109),
 ('experience', 96),
 ('data', 94),
 ('analyst', 87),
 ('product', 85),
 ('skill', 83),
 ('technology', 76),
 ('level', 65),
 ('company', 63),
 ('information', 59),
 ('help', 57),
 ('time', 55),
 ('analysis', 52),
 ('knowledge', 51),
 ('industry', 50),
 ('employment', 50),
 ('new', 46),
 ('language', 46),
 ('accounting', 44),
 ('service', 43),
 ('type', 42),
 ('investment', 42),
 ('role', 41),
 ('ability', 41),
 ('good', 40),
 ('customer', 39)]